In [1]:
!pip install Genderize

In [2]:
!pip install icecream

In [14]:
## import necessary libraries
import pandas as pd
from icecream import ic 
import requests
from genderize import Genderize
import numpy as np

In [4]:
## bring in the cleaned dataset
df = pd.read_excel("cleaned2022_data.xlsx")
df

,name,mangaka,company,medium,month,author1_firstname,author2_firstname,author3_firstname
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao,NaN,NaN
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead,NaN,NaN
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh,NaN,NaN
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji,NaN,NaN
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa,NaN,NaN
...,...,...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao,NaN,NaN
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,",NaN,NaN
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke,NaN,NaN
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika,NaN,NaN


In [5]:
##drop everything after authorfirst name because we only need one author and also because genderize works
##on first name
df = df.drop(df.columns[6:], axis=1) 
df

,name,mangaka,company,medium,month,author1_firstname
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa
...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,"
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika


In [6]:
##rename the column we need to use
df.rename(columns={"author1_firstname": "author_name"}, inplace = True)
df

,name,mangaka,company,medium,month,author_name
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa
...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,"
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika


In [46]:
#did not end up using this

# def gender_certainty(name, min_prob):
#     '''
#     takes a string name and the min_prob, returns gender probability based on using gender_data function
#     '''
#     prob = gender_data(name)[0].get("probability")
#     min_prob = 0.98
#     if prob >= min_prob:
#         return "pass"
#     else:
#         return "flag"

In [7]:
##using genderize now
## FUNCTION to get gender data from genderize
def gender_test(name, api_key):
    '''
    takes a string name and returns gender data
    '''
    converted_to_list = [name]
    gender_data = gd().get(converted_to_list).apply(api_key)
    return gender_info

In [71]:
##THIS DID NOT WORK, SEE BELOW FOR A NEW METHOD

# def gender_count(name):
#     '''
#     takes a string name and the min_prob, returns gender probability based on using gender_data function
#     '''
#     count = gender_data(name)[0].get("count")
#     number = 20_000
#     if count >= number:
#         return "sufficient"
#     else:
#         return "recheck"

In [8]:
## getting the gender 
gender_data = lambda name, api_key:\
requests.get(f'https://api.genderize.io?name={name}&apikey={api_key}').json().get('gender')

In [9]:
def gender_column(df, api_key):
    df['gender'] = df['author_name'].apply(lambda name: gender_data(name, api_key))
    return df

In [10]:
##getting the probability
gender_prob = lambda name, api_key:\
requests.get(f'https://api.genderize.io?name={name}&apikey={api_key}').json().get('probability')

In [11]:
def gender_probs(name, api_key):
    '''
    takes a string name, returns gender probability based on using gender_data function
    '''
    df['gender_prob'] = df['author_name'].apply(lambda name: gender_prob(name, api_key))
    return df

In [12]:
api_key = "enter-api-key"

In [22]:
##the function isnt working, checking to see if i have empty rows of data
df['author_name'] = df['author_name'].astype(object)

In [23]:
##getting the gender 
df = gender_column(df, api_key)
df

,name,mangaka,company,medium,month,author_name,gender
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao,male
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead,male
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh,None
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji,None
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa,male
...,...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao,male
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,",None
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke,male
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika,None


In [24]:
df.to_csv("2022_genderdata.csv", index = False, encoding = "UTF-8")

In [25]:
##getting probability
df = gender_probs(df, api_key)
df

,name,mangaka,company,medium,month,author_name,gender,gender_prob
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao,male,1.00
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead,male,0.67
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh,None,0.00
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji,None,0.00
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa,male,1.00
...,...,...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao,male,0.55
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,",None,0.00
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke,male,1.00
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika,None,0.00


In [26]:
df.to_csv("2022_genderprobability.csv", index = False, encoding = "UTF-8")

In [27]:
df_count = pd.read_csv("2022_genderprobability.csv")
df_count

,name,mangaka,company,medium,month,author_name,gender,gender_prob
0,The Abstract and the Concrete,Isao Hosoya,One Peace Books,Book,january,Isao,male,1.00
1,A Wild Last Boss Appeared! Volume 7 (NOOK Edit...,Firehead,J-Novel Club,Light Novel,january,Firehead,male,0.67
2,10 Dance 6 (NOOK Edition),Inouesatoh,Kodansha International,Digital Edition,january,Inouesatoh,NaN,0.00
3,Akashic Records of Bastard Magic Instructor Vo...,Hitsuji Tarou,Seven Seas Entertainment,Manga,january,Hitsuji,NaN,0.00
4,Akashic Records of Bastard Magic Instructor Vo...,Aosa Tsunemi,Seven Seas Entertainment,Digital Edition,january,Aosa,male,1.00
...,...,...,...,...,...,...,...,...
4321,Grimgar of Fantasy and Ash: Volume 19 (NOOK Ed...,Ao Jyumonji,J-Novel Club,Digital Light Novel,december,Ao,male,0.55
4322,Turning the Tables on the Seatmate Killer! Cha...,"Aresanzui, Miyakobachi",Azuki,Digital Edition,december,"Aresanzui,",NaN,0.00
4323,Natsume & Natsume Chapter 7.1 (Azuki),Shunsuke Sorato,Azuki,Digital Edition,december,Shunsuke,male,1.00
4324,A Favorable Wind on Full Sails,Arantoochika,Glacier Bay Books,Manga,december,Arantoochika,NaN,0.00


In [ ]:
##i couldnt figure out the lambda for gender counts so what i did was:
##1. turned my names into a list
##2. used genderize to get the count
##3. then put the count on a df which then i exported as a csv
##4. then went on the two csvs that i had, one with the probability and gender and the other with the count
##5. then i pasted the count onto the first file, saving it as a new file 

In [28]:
name_list = df_count['author_name'].tolist()
name_list

['Isao',
 'Firehead',
 'Inouesatoh',
 'Hitsuji',
 'Aosa',
 'Natsu',
 'Natsu',
 'Yukito',
 'Shinichi',
 'Shinichi',
 'Ukyo',
 'Ukyo',
 'Yashu',
 'Yashu',
 'Hiroro',
 'Hiroro',
 'Maki',
 'Ryoji',
 'Koyoharu',
 'Akira',
 'Akira',
 'Hiro',
 'Haruki',
 'Haruki',
 'Hiroyuki',
 'Rin',
 'Hamuo',
 'Shiroichi',
 'Yuhki',
 'Yukiya',
 'Yukiya',
 'Yosuke',
 'Kakeru',
 'KAKERU',
 'Ko',
 'Ko',
 'Waco',
 'Waco',
 'Shotaro',
 'Shotaro',
 'You',
 'Io',
 'Io',
 'Kami',
 'Kami',
 'Mito',
 'Mito',
 'Hajime',
 'Hajime',
 'Minto',
 'Ryosuke',
 'Ryosuke',
 'Yuki',
 'Yoko',
 'Yoko',
 'Tetsu',
 'Shinkoshoto',
 'Shinkoshoto',
 'Wataru',
 'Wataru',
 'TNSK',
 'Lynn',
 'Lynn',
 'Yasunori',
 'Daiki',
 'Daiki',
 'Chomoran',
 'Sorata',
 'Sorata',
 'Makoto',
 'Makoto',
 'Yasunori',
 'Yasunori',
 'Yomu',
 'Yoshiaki',
 'Yoshiaki',
 'Yoshifumi',
 'Yoshifumi',
 'Kuji',
 'Kuji',
 'Stan',
 'pixiv',
 'pixiv',
 'Tadahiro',
 'Tadahiro',
 'Natsu',
 'Himataro',
 'Yoshinobu',
 'Gamei',
 'Rhythm',
 'Hikaru',
 'Miya',
 'SHOOWA',
 'S

In [29]:
result = Genderize(api_key = "enter-api-key").get(name_list)
result

[{'count': 1724, 'name': 'Isao', 'gender': 'male', 'probability': 1.0},
 {'count': 3, 'name': 'Firehead', 'gender': 'male', 'probability': 0.67},
 {'count': 0, 'name': 'Inouesatoh', 'gender': None, 'probability': 0.0},
 {'count': 0, 'name': 'Hitsuji', 'gender': None, 'probability': 0.0},
 {'count': 2, 'name': 'Aosa', 'gender': 'male', 'probability': 1.0},
 {'count': 295, 'name': 'Natsu', 'gender': 'male', 'probability': 0.57},
 {'count': 295, 'name': 'Natsu', 'gender': 'male', 'probability': 0.57},
 {'count': 44, 'name': 'Yukito', 'gender': 'male', 'probability': 0.93},
 {'count': 3553, 'name': 'Shinichi', 'gender': 'male', 'probability': 1.0},
 {'count': 3553, 'name': 'Shinichi', 'gender': 'male', 'probability': 1.0},
 {'count': 27, 'name': 'Ukyo', 'gender': 'male', 'probability': 0.74},
 {'count': 27, 'name': 'Ukyo', 'gender': 'male', 'probability': 0.74},
 {'count': 63, 'name': 'Yashu', 'gender': 'male', 'probability': 0.71},
 {'count': 63, 'name': 'Yashu', 'gender': 'male', 'probab

In [32]:
df_result = pd.DataFrame(result)
df_result

,count,name,gender,probability
0,1724,Isao,male,1.00
1,3,Firehead,male,0.67
2,0,Inouesatoh,None,0.00
3,0,Hitsuji,None,0.00
4,2,Aosa,male,1.00
...,...,...,...,...
4321,596,Ao,male,0.55
4322,0,"Aresanzui,",None,0.00
4323,2187,Shunsuke,male,1.00
4324,0,Arantoochika,None,0.00


In [33]:
df_result.to_csv('2022_gendercount.csv')